In [337]:
import pandas as pd
import numpy as np


In [338]:
#Read the train data
train_df = pd.read_csv("../data/train.csv")
# train_df.describe(include="all")

In [339]:
#Read the test data
test_df = pd.read_csv("../data/test.csv")
# test_df.describe(include="all")

In [340]:
#For replacing the NaN Age values we will use the average Age by Sex

#Getting the mean by sex an sorting the dataframe such that it is shows female in index 0 and male in index 1
train_grp = train_df[["Sex","Age"]].groupby(["Sex"], as_index = False).mean().sort_values(by=["Sex"])
train_grp

,Sex,Age
0,female,27.915709
1,male,30.726645


In [341]:
# Replacing the average values for each sex

female_avg_age = train_grp.at[0,"Age"]
male_avg_age = train_grp.at[1,"Age"]

train_df.loc[((train_df["Sex"] == "male") & (np.isnan(train_df["Age"]))), "Age"] = male_avg_age
train_df.loc[((train_df["Sex"] == "female") & (np.isnan(train_df["Age"]))), "Age"] = female_avg_age


In [342]:
#Making the same age replacement for the test file
test_grp = test_df[["Sex","Age"]].groupby(["Sex"], as_index = False).mean().sort_values(by=["Sex"]).sort_values(by=["Sex"])

female_avg_age = test_grp.at[0,"Age"]
male_avg_age = test_grp.at[1,"Age"]

test_df.loc[((test_df["Sex"] == "male") & (np.isnan(test_df["Age"]))), "Age"] = male_avg_age
test_df.loc[((test_df["Sex"] == "female") & (np.isnan(test_df["Age"]))), "Age"] = female_avg_age

In [343]:
# Cabin variable is missing more than 3/4 of total values. Cabin is dropped for the analysis for both train and test sets
# Name is also irrelevant for the analysis, it's dropped

train_df.drop(columns=["Cabin", "Name"], inplace=True)
test_df.drop(columns=["Cabin", "Name"], inplace=True)



In [344]:
# One-hot encoding for Sex
train_one_hot = pd.get_dummies(train_df["Sex"])
train_df.drop(columns=["Sex"], inplace=True)
train_df = train_df.join(train_one_hot)

test_one_hot = pd.get_dummies(test_df["Sex"])
test_df.drop(columns=["Sex"], inplace=True)
test_df = test_df.join(test_one_hot)

In [345]:
# We are only missing some few values in the Embarked variable for training data. We will fill it with the Embarked mode 

train_df["Embarked"] = train_df["Embarked"].fillna(train_df["Embarked"].mode()[0])

In [346]:
#One-hot encoding for the Embarked variable
train_one_hot = pd.get_dummies(train_df["Embarked"])
train_df.drop(columns=["Embarked"], inplace=True)
train_df = train_df.join(train_one_hot)

test_one_hot = pd.get_dummies(test_df["Embarked"])
test_df.drop(columns=["Embarked"], inplace=True)
test_df = test_df.join(test_one_hot)



In [347]:
# Get only the number part of the ticket

train_df["Ticket"] = train_df["Ticket"].apply(lambda x: x.split()[-1])

#Eliminate the nly values without number 
train_df.drop(train_df[train_df["Ticket"] == "LINE"].index , inplace=True)

#Cast as int
train_df["Ticket"] = train_df["Ticket"].astype("int")

In [348]:
#Repeat last change for test data
test_df["Ticket"] = test_df["Ticket"].apply(lambda x: x.split()[-1])
test_df.drop(test_df[test_df["Ticket"] == "LINE"].index , inplace=True)
test_df["Ticket"] = test_df["Ticket"].astype("int")

In [349]:
# Drop the PassengerId, not important for the training model
test_data = test_df.drop(columns=["PassengerId"])
train_data = train_df.drop(columns=["PassengerId"])

pass